<a href="https://colab.research.google.com/github/ThamilezaiAnanthakumar/Cuffless-Blood-Pressure-Monitoring-using-PAT/blob/main/BP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df= pd.read_csv("/content/cleaned_df_imputed.csv")

In [3]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,1105,1106,1107,1108,1109,1110,1111,1112,Systolic BP,Diastolic BP
count,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,...,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000
mean,0.647727,0.676418,0.673703,0.671036,0.673141,0.674988,0.669060,0.667582,0.563398,0.599301,...,0.680313,0.681631,0.682080,0.682104,0.682072,0.682530,0.680916,0.681478,135.785448,65.394543
std,0.110065,0.051301,0.059812,0.058137,0.057723,0.057680,0.061267,0.067151,0.242090,0.201156,...,0.043039,0.042291,0.042057,0.042007,0.042123,0.041935,0.042671,0.042284,22.653566,9.196103
min,0.312000,0.472000,0.464000,0.456000,0.480000,0.432000,0.408000,0.376000,0.040000,0.096000,...,0.472000,0.472000,0.472000,0.472000,0.472000,0.472000,0.472000,0.472000,74.409100,51.071509
25%,0.626000,0.640000,0.640000,0.632000,0.632000,0.640000,0.632000,0.632000,0.584000,0.616000,...,0.648000,0.656000,0.668000,0.668000,0.668000,0.668000,0.656000,0.656000,119.137147,57.858713
50%,0.680000,0.680000,0.680000,0.680000,0.680000,0.680000,0.680000,0.680000,0.672000,0.672000,...,0.680000,0.680000,0.680000,0.680000,0.680000,0.680000,0.680000,0.680000,135.050095,63.834599
75%,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,...,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,0.712000,151.193988,71.930923
max,0.904000,0.824000,0.808000,0.816000,0.816000,0.848000,0.816000,0.816000,1.056000,0.896000,...,0.792000,0.792000,0.792000,0.792000,0.792000,0.792000,0.792000,0.792000,196.490968,93.720710


In [4]:
df.isnull().sum()

,0
0,0
1,0
2,0
3,0
4,0
...,...
1110,0
1111,0
1112,0
Systolic BP,0


In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1105,1106,1107,1108,1109,1110,1111,1112,Systolic BP,Diastolic BP
0,0.712,0.704,0.704,0.712,0.696,0.712,0.704,0.704,0.696,0.704,...,0.712,0.712,0.712,0.712,0.712,0.712,0.712,0.712,144.729100,72.720148
1,0.648,0.632,0.640,0.640,0.648,0.656,0.648,0.656,0.632,0.640,...,0.648,0.648,0.648,0.648,0.648,0.648,0.648,0.648,109.433043,52.947696
2,0.648,0.664,0.648,0.656,0.640,0.632,0.616,0.640,0.672,0.848,...,0.656,0.656,0.656,0.656,0.656,0.656,0.656,0.656,173.209862,73.697431
3,0.720,0.720,0.728,0.720,0.720,0.728,0.720,0.720,0.728,0.728,...,0.704,0.704,0.704,0.704,0.704,0.704,0.704,0.704,135.038979,61.380101
4,0.488,0.632,0.640,0.632,0.624,0.624,0.616,0.392,0.696,0.184,...,0.640,0.640,0.640,0.640,0.640,0.640,0.640,0.640,134.996052,64.722653


In [6]:
feature_cols = df.columns[:-2]

In [7]:
X = df[feature_cols]

In [8]:
y_systolic = df['Systolic BP']
y_diastolic = df['Diastolic BP']

In [9]:

scaler_X = StandardScaler()
scaler_sys = StandardScaler()
scaler_dia = StandardScaler()


X_scaled = scaler_X.fit_transform(X)
y_systolic_scaled = scaler_sys.fit_transform(y_systolic.values.reshape(-1, 1))
y_diastolic_scaled = scaler_dia.fit_transform(y_diastolic.values.reshape(-1, 1))

In [10]:
# Reshape X to 3D for LSTM: (samples, timesteps, features)
X_scaled_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # (samples, timesteps, 1)

In [11]:
X_train, X_test, y_train_s, y_test_s, y_train_d, y_test_d = train_test_split(
    X_scaled_reshaped, y_systolic_scaled, y_diastolic_scaled, test_size=0.2, random_state=42)

In [12]:
!pip install --upgrade scikit-learn scikeras


In [13]:
!pip install scikeras


In [14]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [15]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
import tensorflow.keras.backend as K

# Define a function to create your model (this is required for the wrapper)
def create_model():
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        LSTM(units=50, return_sequences=True, kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        LSTM(units=50, kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(units=1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
    return model

# Wrap your model
regressor = KerasRegressor(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Now perform cross-validation (for example 5 folds)
scores = cross_val_score(regressor, X_scaled_reshaped, y_systolic_scaled, cv=5, scoring='neg_mean_squared_error')

print("Cross-validation MSE scores:", -scores)
print("Average MSE:", -scores.mean())

# Clear backend after training to avoid clutter
K.clear_session()


/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in 

Cross-validation MSE scores: [0.71173692 0.88591226 0.88318556 0.80178074 0.82092366]
Average MSE: 0.8207078271916247
